# Example of EMM for machine learning reweighting using ADMM optimisation

Using the PIMA diabetes dataset we compare the original dataset with labels to the reweighted dataset with artificial labels. We begin by importing the PIMA diabetes dataset, it is already cleaned and does not have any missing values, outliers, etc. This example is also nice since all features are continuous and the labels are binary.

In [ ]:
# Get raw data, convert to csv, and import
# Update directory
import sys
sys.path.insert(1, '../')
# Import libraries
from emm import *
import os.path
from get_data import get_data
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

# Data locations
raw_dir = "../data/raw/pima_diabetes/archive.zip"
processed_dir = "../data/processed/pima_diabetes/"

# If processed data does exist convert raw to csv
if not os.path.exists(processed_dir + "diabetes.csv"):
    get_data.RData_to_csv(raw_dir,processed_dir)
    
# Get data into dataframe
df = pd.read_csv(processed_dir + "diabetes.csv")

# Replace 0 with nan
nan_cols = ['Glucose', 'BloodPressure','SkinThickness','Insulin','BMI']
df[nan_cols]=df[nan_cols].replace({'0':np.nan,0:np.nan})

Separating features from labels, useful for training ML models and for re-weighting.

In [ ]:
#train_test_splitting of the dataset
X = df.drop(columns = 'Outcome')
# Getting Predicting Value
y = df['Outcome']

First, we look at the means for each features according to outcome.

In [ ]:
marginals_mean = df.groupby('Outcome').mean()
display(marginals_mean)

We may also want to consider standard deviation of features.

In [ ]:
marginals_std = df.groupby('Outcome').std()
display(marginals_std)

### Matching marginals

First we construct functions $F$ upon which we take expectations of under the weighted measure. Mathematically, we express this as
$$ \mathbb E[F(x)] = \sum^N_{i=1} w_i F(x_i) $$

In [ ]:
features = list(X)
marginals = {}
for feature in features:
    marginals[feature] = ['mean', 'std']
marginals

In [ ]:
loss_0 = []
loss_1 = []
for feature in marginals.keys():
    for fun in marginals[feature]:
        marg = getattr(df[[feature,'Outcome']].groupby('Outcome'),fun)()
        loss_0.append(EqualityLoss(marg.loc[0]))
        loss_1.append(EqualityLoss(marg.loc[1]))

In [ ]:
regularizer = regularizers.EntropyRegularizer(limit=None)
#regularizer = regularizers.ZeroRegularizer()
w_0, out_0, sol_0 = emm(X, marginals, loss_0, regularizer, 
                        optimizer = 'gurobi', verbose=False, rho=25, 
                        eps_abs=1e-6, eps_rel=1e-6)
w_1, out_1, sol_1 = emm(X, marginals, loss_1, regularizer,
                     optimizer='gurobi', verbose=False, rho=24,
                        eps_abs=1e-6, eps_rel=1e-6)

In [ ]:
X_0 = X.copy()
X_0["weights"] = w_0 
X_1 = X.copy()
X_1["weights"] = w_1 

# Set theoretical outcome to train on reweighted datasets
X_0['Outcome'] = 0
X_1['Outcome'] = 1

df_w = pd.concat([X_0,X_1])

In [ ]:
df_valw = df_w.drop(columns=['Outcome','weights']).multiply(df_w['weights'], axis="index")
df_valw['Outcome'] = df_w['Outcome']

In [ ]:
w_marginals = df_valw.groupby('Outcome').sum()

In [ ]:
display(marginals_mean)
display(w_marginals)
display(abs((marginals_mean - w_marginals)))

In [ ]:
std_w = pd.concat([abs(X_0[features] - marginals_mean.loc[0]),abs(X_1[features] - marginals_mean.loc[1])])
std_w = std_w.multiply(df_w['weights'], axis="index")
std_w = pd.concat([std_w, df_w['Outcome']],axis=1)

In [ ]:
display(marginals_std)
display(std_w.groupby('Outcome').sum())
display(abs((marginals_std - std_w.groupby('Outcome').sum())))